### Compass Files

In [ ]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
import numpy as np
from evFileProcessing import evFuncs # ev COM functions

In [ ]:
def readCompassPMEL(files, mooring, start):
    dir = []
    temp = []
    time = []
    for file in files:
        try:
            dfCompass=pd.read_csv(file, header = None)
            for dat in dfCompass[1]:
                dir.append(float(dat.split('P')[0][2:]))
                temp.append(float(dat.split('-')[-1][:-3]))
            for t in dfCompass[0]:
                time.append(t)
        except:
            continue
    corrTime = [start+(x- pd.to_datetime(time[5])) for x in  pd.to_datetime(time[5:min(len(time), len(dir))])]
    df = pd.DataFrame({'Datetime':corrTime,'Dir_'+str(mooring):dir[5:min(len(time), len(dir))]})
    df = df.set_index('Datetime')
    return df
    return time, dir

def readCompassAaronia(file, mooring):
    df = pd.read_csv(file)
    df['Datetime'] = pd.to_datetime(df['date'] + ' ' + df['time(local time)'])
    df = df[['Datetime','compass (deg)']]
    df = df.rename(columns={'compass (deg)':'Dir_'+str(mooring)})
    df = df.set_index('Datetime')
    return df

def readCompassOpenTag(files, mooring):
    dfs = []
    for file in files:
        df = pd.read_csv(file)
        dfs.append(df)
        dfA = pd.concat(dfs)
    dfA = dfA.reset_index()
    return dfA

dfD1Y1C1 = readCompassPMEL(glob('../data/2018/DAFT1/Compass/PMEL/SERIAL/*.txt'), 1, datetime.datetime(2017,8,6,17,0,0))
dfD1Y1C2 = readCompassOpenTag(glob('../data/2018/DAFT1/Compass/OpenTag/*.csv'), 1)
dfD2Y1C1 = readCompassPMEL(glob('../data/2018/DAFT2/Compass/PMEL/SERIAL/*.txt'), 2, datetime.datetime(2017,8,8,6,40,0))
dfD1Y2C1 = readCompassAaronia('../data/2019//DAFT1/Compass/Aaronia/DSFT1Converted_Formatted.csv',1)
dfD2Y2C1 = readCompassPMEL(glob('../data/2019//DAFT2/Compass/PMEL/SERIAL/*.txt'), 2, datetime.datetime(2018,8,12,16,20,0))
dfD3Y2C1 = readCompassPMEL(glob('../data/2019//DAFT3/Compass/PMEL/SERIAL/*.txt'), 3, datetime.datetime(2018,8,14,19,30,0))

ds = [dfD1Y1C1,dfD1Y2C1,dfD2Y1C1,dfD2Y2C1,dfD3Y2C1]
fnames = ['C11_2017-2018_CompassHeading.csv','C11_2018-2019_CompassHeading.csv',
 'C1_2017-2018_CompassHeading.csv','C1_2018-2019_CompassHeading.csv',
 'C4_2018-2019_CompassHeading.csv']
magDec =  [9.6, 10.3, 11.6,12.2,13.4]
ct = 0
for d in ds:
    d = d.rename(columns ={d.columns[0]:'Degrees'})
    d['Magnetic Declination'] = magDec[ct]
    d.to_csv(fnames[ct])
    ct+=1


## Mooring Files

In [ ]:
# This builds out the axiom csv files
fnames = ['C11_2017-2018_WBAT','C11_2018-2019_WBAT',
             'C1_2017-2018_WBAT','C1_2018-2019_WBAT',
             'C4_2018-2019_WBAT']
deployments = [['DAFT1','2018'],['DAFT1','2019'],['DAFT2','2018'],['DAFT2','2019'],['DAFT3','2019']]
ct =0

for d in deployments:
    df70 = evFuncs.evExports.readEvExports(sorted(glob('E:/MooredEchosounders/data/'+d[1]+'/EVFiles/'+d[0]+'/exports/1m/70*.csv')))
    df38 = evFuncs.evExports.readEvExports(sorted(glob('E:/MooredEchosounders/data/'+d[1]+'/EVFiles/'+d[0]+'/exports/1m/38*.csv')))
    df200 = evFuncs.evExports.readEvExports(sorted(glob('E:/MooredEchosounders/data/'+d[1]+'/EVFiles/'+d[0]+'/exports/1m/200*.csv')))
   
    df70['DateTime'] = pd.to_datetime(df70.index.map(lambda x: x.strftime('%Y-%m-%d %H')))
    df70_clean = df70.set_index(['DateTime','layer'])
    df38['DateTime'] = pd.to_datetime(df38.index.map(lambda x: x.strftime('%Y-%m-%d %H')))
    df38_clean = df38.set_index(['DateTime','layer'])
    df200['DateTime'] = pd.to_datetime(df200.index.map(lambda x: x.strftime('%Y-%m-%d %H')))
    df200_clean = df200.set_index(['DateTime','layer'])
    if (d[0] == 'DAFT1')&(d[1]=='2019'):
        df38_clean['sA'] = np.nan
    df70_clean['sA'].to_csv('AxiomData/'+fnames[ct]+'_70kHz.csv')
    df38_clean['sA'].to_csv('AxiomData/'+fnames[ct]+'_38kHz.csv')
    df200_clean['sA'].to_csv('AxiomData/'+fnames[ct]+'_200kHz.csv')
    ct+=1

In [ ]:
# This can be used to compare the exports and the packaged axiom datasets
f, (ax1)= plt.subplots(1,1,figsize=(20,8),sharex=True, sharey=True,dpi=150)

df1 = evFuncs.evExports.readEvExports(sorted(glob('E:/MooredEchosounders/data/2018/EVFiles/'+daft+'/exports/1m/38*.csv'))+sorted(glob('E:/MooredEchosounders/data/2019/EVFiles/'+daft+'/exports/1m/38*.csv')))
ax1=plt.subplot(311)
a = df1['sA'].resample('2H').sum()
a.plot( style='.',label='Wakeups',legend=False,axes=ax1, color='tomato')
a.resample('D').mean().plot(legend=False,label='Daily Mean', axes=ax1,linewidth=2,color='k')

#ax1.set_ylim([-60,1000])
#ax1.set_yticks([0,1000,2000,3000])
ax1.set_xlim([pd.to_datetime('2017-8-7'),pd.to_datetime('2019-9-4')])
ax1.grid()
ax1.legend(['Wakeup Mean','Daily Mean'])#,loc = 'lower left')
#ax1.set_xticklabels([])
#ax1.xaxis.label.set_visible(False)
plt.ylabel('70 kHz Mean Sv')

df38_clean.reset_index().set_index('DateTime').resample('2H').sum().sA.plot(ax=ax1)